# Types of Gradient Descent
## Batch GD
In calculating the slope, we use the entire data available to provide an update. As a result, this approach is quite slow. This is effective for convex functions, given the data size is not too large.
## Stochastic GD
Stochastic GD works on the basis of a single row, and update the slope accordingly. This is fast, but is prone to errors. This approach is usually preferred.
## Mini-Batch GD
The data is split into mini-batches and the slope is updated after every mini-batch.

# Coding
## Batch GD

In [1]:
from sklearn.datasets import load_diabetes
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [2]:
X,y = load_diabetes(return_X_y=True)

In [3]:
print(X.shape)
print(y.shape)

(442, 10)
(442,)


In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [5]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [9]:
m = reg.coef_
b = reg.intercept_
print('Coefficients: ',m,'\n\nIntercept: ',b)

Coefficients:  [  -23.57255297  -248.38154175   520.32454423   322.74874835
 -1192.03298849   759.07622019   304.4451743    345.09263455
   842.42001446    93.92153769] 

Intercept:  149.1733966836802


In [10]:
y_pred = reg.predict(X_test)

In [11]:
r2_score(y_test,y_pred)

0.42500266019187105

In [62]:
class GDRegressor:
    def __init__(self,learning_rate=0.01,epochs=100):
        self.coef_=None
        self.intercept_=None
        self.lr = learning_rate
        self.epochs = epochs

    def fit(self,X_train,y_train):
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        for i in range(self.epochs):
            y_hat = np.dot(X_train,self.coef_) + self.intercept_
            intercept_der = -2 * np.mean(y_train-y_hat)
            coef_der = -2 * np.dot((y_train -y_hat),X_train)/X_train.shape[0]
            self.intercept_ -= self.lr* intercept_der
            self.coef_ -= self.lr * coef_der
        
        print(self.intercept_,self.coef_)
        
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [63]:
gdr = GDRegressor(epochs=300)

In [64]:
gdr.fit(X_train,y_train)

152.56951726379626 [ 5.06275923  1.91792834 14.98798578 10.79043187  5.26184667  4.82009969
 -8.095976   10.75241987 13.07073288  9.51723004]


## Stochastic Gradient Descent
#### Why use Stochastic GD?
<ul>
    <li>Faster convergence</li>
    <li>Not computationally expensive</li>
    <li>Stochastic GD is more likely to not converge to a local minima</li>
</ul> However, you may not obtain a steady solution. To overcome this issue, we can employ learning schedule.

#### When to use Stochastic GD?
<ul>
    <li>Big Data</li>
    <li>Non-Convex Function </li>
</ul>

In [65]:
X,y = load_diabetes(return_X_y=True)

In [66]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [67]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [68]:
print(reg.coef_)
print(reg.intercept_)

[  -9.15865318 -205.45432163  516.69374454  340.61999905 -895.5520019
  561.22067904  153.89310954  126.73139688  861.12700152   52.42112238]
151.88331005254167


In [71]:
class SGDRegressor:
    def __init__(self,learning_rate=0.01,epochs=100):
        self.coef_=None
        self.intercept_=None
        self.lr = learning_rate
        self.epochs = epochs

    def fit(self,X_train,y_train):
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        for i in range(self.epochs):
            for j in range(X_train.shape[0]):
                idx = np.random.randint(0,X_train.shape[0])
                y_hat = np.dot(X_train[idx],self.coef_)+self.intercept_

                intercept_der = -2 * (y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)

                coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                self.coef_ -= self.lr * coef_der
        print(self.intercept_,self.coef_)
        
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [76]:
sgd = SGDRegressor(epochs=100)

In [77]:
sgd.fit(X_train,y_train)

153.17745679017563 [  30.18697498 -139.84461442  453.45122124  309.11207294  -27.26300629
  -95.81719191 -197.32076594  111.27092809  412.02084356  116.40030495]


In [78]:
y_pred = sgd.predict(X_test)

In [79]:
r2_score(y_test,y_pred)

0.4527184479370052

## Mini-Batch Gradient Descent
Given data, we split the rows into batches and then make updates accordingly.

In [80]:
from sklearn.linear_model import LinearRegression

In [81]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [82]:
m = reg.coef_
b = reg.intercept_
print(m,b)

[  -9.15865318 -205.45432163  516.69374454  340.61999905 -895.5520019
  561.22067904  153.89310954  126.73139688  861.12700152   52.42112238] 151.88331005254167


In [83]:
y_pred = reg.predict(X_test)
r2_score(y_test,y_pred)

0.4399338661568969

In [89]:
class MBGDRegressor:
    
    def __init__(self,batch_size,learning_rate=0.01,epochs=100):
        
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        
    def fit(self,X_train,y_train):
        # init your coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range(self.epochs):
            
            for j in range(int(X_train.shape[0]/self.batch_size)):
                
                idx = random.sample(range(X_train.shape[0]),self.batch_size)
                
                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
                #print("Shape of y_hat",y_hat.shape)
                intercept_der = -2 * np.mean(y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)

                coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_der)
        
        print(self.intercept_,self.coef_)
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [92]:
mbr = MBGDRegressor(batch_size=int(X_train.shape[0]/50),learning_rate=0.01,epochs=100)
mbr.fit(X_train,y_train)

150.02589444260533 [  30.68213703 -134.47622368  461.79621425  305.65058559  -24.03032736
  -95.72198025 -187.52849647  106.63125379  408.01766928  113.23412313]


In [94]:
y_pred = mbr.predict(X_test)

In [96]:
r2_score(y_test,y_pred)

0.4511943037483286